### CNN on clean dataset

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded= files.upload()

Saving cnn_model.py to cnn_model.py


In [ ]:
from cnn_model import ECGCNN

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ECG_Project/preprocessed/ecg_clean.csv')

In [ ]:
X = df.drop('label', axis=1).values
y = df['label'].astype('category').cat.codes.values

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

Train: torch.Size([70023, 216]) Val: torch.Size([15005, 216]) Test: torch.Size([15005, 216])


In [ ]:
classes = np.array(list(set(y.numpy())))
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y.numpy())
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
print("Class weights:", class_weights_tensor)

Class weights: tensor([7.8581, 2.4782, 0.2667, 2.7569, 2.8064], device='cuda:0')


In [ ]:
num_classes = len(classes)
model = ECGCNN(num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 20
batch_size = 64

train_dataset = TensorDataset(X_train, y_train)
val_dataset   = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}]  Train Loss: {train_loss:.4f}  Train Acc: {train_acc:.4f}  Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), '/content/drive/MyDrive/ECG_Project/cnn_clean_best.pth')
        print("Best model saved ✅")

Epoch [1/20]  Train Loss: 0.2426  Train Acc: 0.9105  Val Acc: 0.9234
Best model saved ✅
Epoch [2/20]  Train Loss: 0.1307  Train Acc: 0.9501  Val Acc: 0.9744
Best model saved ✅
Epoch [3/20]  Train Loss: 0.1036  Train Acc: 0.9610  Val Acc: 0.9771
Best model saved ✅
Epoch [4/20]  Train Loss: 0.0858  Train Acc: 0.9675  Val Acc: 0.9831
Best model saved ✅
Epoch [5/20]  Train Loss: 0.0775  Train Acc: 0.9708  Val Acc: 0.9856
Best model saved ✅
Epoch [6/20]  Train Loss: 0.0691  Train Acc: 0.9746  Val Acc: 0.9661
Epoch [7/20]  Train Loss: 0.0636  Train Acc: 0.9764  Val Acc: 0.9819
Epoch [8/20]  Train Loss: 0.0540  Train Acc: 0.9788  Val Acc: 0.9835
Epoch [9/20]  Train Loss: 0.0517  Train Acc: 0.9804  Val Acc: 0.9836
Epoch [10/20]  Train Loss: 0.0440  Train Acc: 0.9820  Val Acc: 0.9764
Epoch [11/20]  Train Loss: 0.0409  Train Acc: 0.9842  Val Acc: 0.9865
Best model saved ✅
Epoch [12/20]  Train Loss: 0.0362  Train Acc: 0.9844  Val Acc: 0.9883
Best model saved ✅
Epoch [13/20]  Train Loss: 0.0342  T

In [ ]:
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model.load_state_dict(torch.load('/content/drive/MyDrive/ECG_Project/cnn_clean_best.pth'))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.9913
